In [1]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-4 

Cloning into 'ATML-PA-4'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 249 (delta 175), reused 155 (delta 86), pack-reused 0 (from 0)
Receiving objects: 100% (249/249), 390.02 KiB | 14.45 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [50]:
!git pull  http://github.com/Zapy67/ATML-PA-4 atm-10

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 3.23 KiB | 661.00 KiB/s, done.
From http://github.com/Zapy67/ATML-PA-4
 * branch            atm-10     -> FETCH_HEAD
Updating b08e68f..b6ed31c
Fast-forward
 fed_lib/fed_methods.py | 413 ++++++++++++++++++++++++++++++++-----------------
 fedsam.ipynb           | 316 ++++++++++++++++++++++++++++++-------
 2 files changed, 531 insertions(+), 198 deletions(-)


In [2]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [3]:
!ls

'ATML - PA 4.pdf'   fed_lib	   LICENSE     task1.ipynb   task3.ipynb
 data		    fedsam.ipynb   README.md   task2.ipynb   task4.ipynb


#  FedSAM on heterogeneous domains

In [51]:
import torch
import torch.nn as nn

In [52]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [6]:
!pip install fedlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:

In [53]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

ModuleNotFoundError: No module named 'utils'

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
trainset, testset = utils.get_cifar10()

In [24]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [26]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.039 0.197 0.102 0.121 0.082 0.167 0.02  0.031 0.115 0.126]
Entropy of Label Distribution H(P(Y)): 2.0409007076773857
Normalized Entropy of Label Distribution: 0.8863519154567302

Label Distribution (P(Y)): [0.082 0.079 0.135 0.053 0.128 0.1   0.034 0.144 0.244]
Entropy of Label Distribution H(P(Y)): 1.9797306458058213
Normalized Entropy of Label Distribution: 0.9010142459838699

Label Distribution (P(Y)): [0.042 0.035 0.101 0.016 0.078 0.123 0.207 0.043 0.072 0.283]
Entropy of Label Distribution H(P(Y)): 1.9221358831956668
Normalized Entropy of Label Distribution: 0.8347730075401114

Label Distribution (P(Y)): [0.112 0.132 0.102 0.209 0.087 0.025 0.151 0.182]
Entropy of Label Distribution H(P(Y)): 1.8967389565875812
Normalized Entropy of Label Distribution: 0.9121386288432701

Label Distribution (P(Y)): [0.242 0.035 0.055 0.092 0.128 0.076 0.104 0.105 0.063 0.1  ]
Entropy of Label Distribution H(P(Y)): 2.0809519365200044
Normalized Entropy of Label Distrib

In [27]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [28]:
client_weights

[0.22526, 0.20142, 0.1886, 0.20208, 0.18264]

In [45]:
def fed_avg_exp(partition: str = 'dirichlet', alpha:float = 0.1, num_clients:int =5, fed_method: fed_methods.FedMethod = None, rho:float = 0.1):
    
    fed = fed_model.Federation(num_clients, None, partition, None, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

    client_loaders = fed.client_dataloaders
    client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]

    fed_method = fed_methods.FedSAM(sam_rho=rho, 
            num_local_steps=int(len(trainset)/num_clients), 
            client_aggregation_weights= client_weights)
    
  
    fed.set_method(fed_method)
    fed.train(25, lr=1e-3, verbose=False, momentum =0, weight_decay=0)



In [ ]:
alphas = [0.1, 0.5, 1, 5, 10, 50, 100]
for alpha in alphas:
    fed_avg_exp(num_clients = 5, alpha=alpha)


--- Round 1/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: 161.6193, Test Acc: 10.09%

--- Round 2/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 2:


FedSAM  | Test Loss: 142.2099, Test Acc: 20.16%

--- Round 3/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 3:


FedSAM  | Test Loss: 122.6762, Test Acc: 29.86%

--- Round 4/25 ---
Training Clients
Training Client 1/5


...

FedSAM  | Test Loss: 34.4928, Test Acc: 81.63%

--- Round 24/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 24:


FedSAM  | Test Loss: 34.2206, Test Acc: 81.98%

--- Round 25/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 25:


FedSAM  | Test Loss: 33.9069, Test Acc: 82.19%
Training Complete!


FedSAM  | Test Loss: 33.9069, Test Acc: 82.19%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                5.187540e+00 6.388851e-01
backbone.2.block.0.weight                4.885939e+00 6.113057e-01
backbone.1.block.0.weight                4.325180e+00 7.034668e-01
backbone.1.block.3.weight                4.235101e+00 6.932738e-01
backbone.0.block.3.weight                4.172994e+00 7.846729e-01
Overall Statistics:
Total L2 difference:        1.177553e+01
Total parameters:           551,466


In [49]:
alphas = [0.1, 0.5, 1, 5, 10, 50, 100]
for alpha in alphas:
    fed_avg_exp(num_clients=5, alpha=alpha ,rho=1e-2)


--- Round 1/25 ---
Training Clients
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: 150.1479, Test Acc: 10.89%

--- Round 2/25 ---
Training Clients
Training Server
Evaluate on round 2:


FedSAM  | Test Loss: 120.1183, Test Acc: 34.19%

--- Round 3/25 ---
Training Clients


Training Server
Evaluate on round 3:


FedSAM  | Test Loss: 103.7813, Test Acc: 43.46%

--- Round 4/25 ---
Training Clients
Training Server
Evaluate on round 4:


...

Training Server
Evaluate on round 18:


FedSAM  | Test Loss: 57.0280, Test Acc: 69.54%

--- Round 19/25 ---
Training Clients


Training Server
Evaluate on round 19:


FedSAM  | Test Loss: 54.5890, Test Acc: 71.18%

--- Round 20/25 ---
Training Clients


KeyboardInterrupt: 